In [78]:
import pandas as pd
from clickhouse_driver import Client

In [79]:
client = Client(host="localhost")

In [80]:
client.execute("CREATE DATABASE IF NOT EXISTS test_db")

[]

In [81]:
client.execute(
    "CREATE TABLE IF NOT EXISTS test_db.test_table_with_null ("
    "    Id               UInt64  NOT NULL,"
    "    EmployeeName     Nullable(TEXT),"
    "    JobTitle         Nullable(TEXT),"
    "    BasePay          Nullable(Float64),"
    "    OvertimePay      Nullable(Float64),"
    "    OtherPay         Nullable(Float64),"
    "    Benefits         Nullable(String),"
    "    TotalPay         Nullable(Float64),"
    "    TotalPayBenefits Nullable(Float64),"
    "    Year             Nullable(UInt32),"
    "    Agency           Nullable(String),"
    "    Status           Nullable(String)"
    ") Engine=MergeTree() ORDER BY Id PRIMARY KEY Id"
)

[]

In [82]:
df = pd.read_csv("./data/sf_salaries.csv", delimiter=",", encoding="utf-8", low_memory=False)

In [83]:
df = df.drop('Notes', axis=1)
df["EmployeeName"] = df["EmployeeName"].replace("Not Provided", None)
df["JobTitle"] = df["JobTitle"].replace("Not Provided", None)
df["BasePay"] = df["BasePay"].replace("Not Provided", None)
df["OvertimePay"] = df["OvertimePay"].replace("0.0", None)
df["OvertimePay"] = df["OvertimePay"].replace("Not Provided", None)
df["OtherPay"] = df["OtherPay"].replace("0.0", None)
df["OtherPay"] = df["OtherPay"].replace("Not Provided", None)
df["Benefits"] = df["Benefits"].replace("0.0", None)
df["Benefits"] = df["Benefits"].replace("0.00", None)
df["Benefits"] = df["Benefits"].replace("Not Provided", None)
data: list = df.values.tolist()

In [84]:
client.insert_dataframe(
    query="INSERT INTO test_db.test_table_with_null VALUES",
    dataframe=pd.DataFrame(
        data=data,
        columns=["Id", "EmployeeName", "JobTitle", "BasePay", "OvertimePay", "OtherPay", "Benefits", "TotalPay", "TotalPayBenefits", "Year", "Agency", "Status"]
    ),
    settings=dict(use_numpy=True)
)

148654

In [101]:
%%timeit
client.execute("SELECT * FROM test_db.test_table_with_null LIMIT 1")

3.38 ms ± 128 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [109]:
%%timeit
client.execute("SELECT count(*) FROM test_db.test_table_with_null WHERE OvertimePay > 30000")

2.67 ms ± 80.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [110]:
client.execute("ALTER TABLE test_db.test_table_with_null ADD INDEX IdxOvertimePay OvertimePay TYPE bloom_filter GRANULARITY 8192")


[]

In [125]:
%%timeit
client.execute("SELECT count(*) FROM test_db.test_table_with_null WHERE OvertimePay > 30000")

2.84 ms ± 52.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
